# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [1]:
# Imports here
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler

from torchvision import transforms, datasets, models

from collections import OrderedDict
from PIL import Image
import json

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). You can [download the data here](https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip). The dataset is split into two parts, training and validation. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. If you use a pre-trained network, you'll also need to make sure the input data is resized to 224x224 pixels as required by the networks.

The validation set is used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks available from `torchvision` were trained on the ImageNet dataset where each color channel was normalized separately. For both sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.

In [2]:
data_dir = 'flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

batch_size = 32
num_workers = 0

In [3]:
# TODO: Define your transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.RandomResizedCrop(224),
                                      transforms.RandomRotation(30),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                          [0.229, 0.224, 0.225])])

validation_transforms = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406],
                                                          [0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder
train_dataset = datasets.ImageFolder(train_dir, transform = train_transforms)
validation_dataset = datasets.ImageFolder(valid_dir, transform = validation_transforms)

test_fraction = 0.1
train_size = len(train_dataset)
indices = list(range(train_size))
np.random.shuffle(indices)
split_idx = int(np.floor(train_size*test_fraction))
train_indices, test_indices = indices[split_idx:], indices[:split_idx]

train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

# TODO: Using the image datasets and the trainforms, define the dataloaders
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size,
                                          sampler = train_sampler, num_workers = num_workers)
testloader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size,
                                        sampler = test_sampler, num_workers = num_workers)
validationloader = torch.utils.data.DataLoader(validation_dataset, batch_size = batch_size,
                                              num_workers = num_workers)

In [7]:
print(len(trainloader), len(testloader), len(validationloader))

185 21 26


In [8]:
print(len(train_dataset), len(validation_dataset))

6552 818


### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [5]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

In [11]:
#model = models.densenet121(pretrained = True) #83%
model = models.densenet201(pretrained = True)

C:\Users\Naveen\Anaconda3\lib\site-packages\torchvision\models\densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
C:\Users\Naveen\Anaconda3\lib\site-packages\torchvision\models\densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
C:\Users\Naveen\Anaconda3\lib\site-packages\torchvision\models\densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
C:\Users\Naveen\Anaconda3\lib\site-packages\torchvision\models\densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
C:\Users\Naveen\Anaconda3\lib\site-packages\torchvision\models\densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.ka

C:\Users\Naveen\Anaconda3\lib\site-packages\torchvision\models\densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
C:\Users\Naveen\Anaconda3\lib\site-packages\torchvision\models\densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
C:\Users\Naveen\Anaconda3\lib\site-packages\torchvision\models\densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
C:\Users\Naveen\Anaconda3\lib\site-packages\torchvision\models\densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
C:\Users\Naveen\Anaconda3\lib\site-packages\torchvision\models\densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.ka

In [12]:
print(model)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [6]:
# TODO: Build and train your network

for param in model.parameters():
    param.requires_grad = False
    
classifier = nn.Sequential(OrderedDict([("fc1", nn.Linear(1024, 500)),
                                         ("relu", nn.ReLU()),
                                         ("drop", nn.Dropout(0.3)),
                                         ("fc2", nn.Linear(500, 102)),
                                         ("output", nn.LogSoftmax(dim=1))]))

model.classifier = classifier

C:\Users\Naveen\Anaconda3\lib\site-packages\torchvision\models\densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
C:\Users\Naveen\Anaconda3\lib\site-packages\torchvision\models\densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
C:\Users\Naveen\Anaconda3\lib\site-packages\torchvision\models\densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
C:\Users\Naveen\Anaconda3\lib\site-packages\torchvision\models\densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
C:\Users\Naveen\Anaconda3\lib\site-packages\torchvision\models\densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.ka

In [7]:
# Validation pass function
def validation(model, validationloader, criterion, device):
    loss = 0
    accuracy = 0
    for inputs, labels in validationloader:
        
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = model.forward(inputs)
        loss += criterion(outputs, labels)
        
        prob = torch.exp(outputs)
        equality = (prob.max(dim=1)[1] == labels)
        accuracy += equality.type(torch.FloatTensor).mean() # Mean of no. of correct predictions
        
    return loss/len(validationloader), accuracy*100/len(validationloader) # Avg. of sum of losses & % accuracy

In [8]:
# Initializations
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr = 0.01)

In [10]:
# Training classifier
model.to(device)

epochs = 7
print_every = 20
steps = 0

for e in range(epochs):
    model.train()
    
    running_loss = 0
    for inputs, labels in trainloader:
        steps += 1
        
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        #Training step
        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % print_every == 0:
            
            # Putting model in evaluation mode
            model.eval()
        
            # Turning gradients off since no need to backpropagate for validation purpose
            with torch.no_grad():
                valid_loss, valid_accuracy = validation(model, validationloader, criterion, device)
                
            print("Epoch: {} of {}    Training Loss: {:.3f}    Validation Loss: {:.3f}    Validation Accuracy: {:.2f} %"
                  .format(e+1, epochs, running_loss/print_every, valid_loss, valid_accuracy))
            
            running_loss = 0
            
            # Putting model back in training mode
            model.train()

print("\nDONE!")

Epoch: 1 of 7    Training Loss: 1.271    Validation Loss: 0.791    Validation Accuracy: 82.60 %
Epoch: 1 of 7    Training Loss: 1.185    Validation Loss: 0.802    Validation Accuracy: 78.10 %
Epoch: 1 of 7    Training Loss: 1.196    Validation Loss: 0.788    Validation Accuracy: 77.76 %
Epoch: 1 of 7    Training Loss: 1.185    Validation Loss: 0.840    Validation Accuracy: 78.02 %
Epoch: 1 of 7    Training Loss: 1.121    Validation Loss: 0.765    Validation Accuracy: 78.59 %
Epoch: 1 of 7    Training Loss: 1.249    Validation Loss: 0.753    Validation Accuracy: 80.33 %
Epoch: 1 of 7    Training Loss: 1.150    Validation Loss: 0.718    Validation Accuracy: 81.50 %
Epoch: 1 of 7    Training Loss: 1.043    Validation Loss: 0.666    Validation Accuracy: 83.76 %
Epoch: 1 of 7    Training Loss: 1.085    Validation Loss: 0.687    Validation Accuracy: 81.98 %


KeyboardInterrupt: 

In [ ]:
total = 0
correct = 0
with torch.no_grad():
    for inputs, labels in testloader:
        total += labels.size(0)
        
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model.forward(inputs)
        probabilities = torch.exp(outputs)
        
        correct += ((probabilities.max(dim=1)[1] == labels).sum().item())

print("Accuracy: {:.2f}%".format(correct*100/total))

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# TODO: Save the checkpoint 
checkpoint = {'input_size': model.classifier[0].in_features,
             'output_size': model.classifier[3].out_features,
             'hidden_layers': [model.classifier[i].out_features for i in (0,3)],
             'drop_probability': model.classifier[2].p,
             'classifier_state_dict': model.classifier.state_dict(),
             'optim_state_dict': optimizer.state_dict(),
             'epochs':epochs,
             'map_class_to_idx': train_dataset.class_to_idx}

torch.save(checkpoint, 'checkpoint.pth')

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_model(path):
    checkpoint = torch.load(path)
    classifier = nn.Sequential(OrderedDict([("fc1", nn.Linear(checkpoint['input_size'], checkpoint['hidden_layers'][0])),
                                         ("relu", nn.ReLU()),
                                         ("drop", nn.Dropout(checkpoint['drop_probability'])),
                                         ("fc2", nn.Linear(checkpoint['hidden_layers'][0], checkpoint['hidden_layers'][1])),
                                         ("output", nn.LogSoftmax(dim=1))]))
    
    classifier.load_state_dict(checkpoint['classifier_state_dict'])
    model = models.densenet121(pretrained = True)
    model.classifier = classifier
    model.class_to_idx = checkpoint['map_class_to_idx']
    
    return model

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
# Normalizes the image array expecting color channel to be first dimension
def normalize_image_array(img_array, means, std):
    ''' Encodes the color channels between 0 and 1 & normalizes the 
        img_array (expecting color channel to be first dimension)
        given the means - 'means' and standard deviations - 'std'
    '''
    img_array = img_array/255
    # For each color channel
    for i in range(img_array.shape[0]):
        img_array[i] = (img_array[i] - means[i])/std[i]
        
    return img_array

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    pil_img = Image.open(image)
    
    # TODO: Process a PIL image for use in a PyTorch model
    
    # Resizing while preserving aspect ratio
    width, height = pil_img.size
    if width <= height:
        new_width = 256
        new_height = int((new_width*height)/width)
    else:
        new_height = 256
        new_width = int((new_height*width)/height)
    pil_img = pil_img.resize(size = (new_width, new_height))
    
    # Center cropping 224x224
    size = pil_img.size
    pil_img = pil_img.crop(( size[0]//2 - 112,
                            size[1]//2 - 112,
                            size[0]//2 + 112,
                            size[1]//2 + 112))
    
    # Converting to numpy array and noramlizing
    np_img = np.array(pil_img, dtype = 'float')
    np_img = np_img.T
    np_img = normalize_image_array(np_img, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    
    return np_img

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    model.eval()
    # TODO: Implement the code to predict the class from an image file
    np_img = process_image(image_path)
    
    tens_img = torch.from_numpy(np_img)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    tens_img = tens_img.to(device)
    tens_img.unsqueeze_(0)
    tens_img = tens_img.type(torch.FloatTensor)
    
    with torch.no_grad():
        output = model.forward(tens_img)
        
    prob = torch.exp(output)
    topk_values, indices = prob.topk(topk)
    
    # Get classes from indices
    indices = indices.numpy().squeeze()
    class_to_index = model.class_to_idx
    index_to_class = {v:k for k,v in class_to_index.items()}
    classes = [int(index_to_class[index]) for index in indices]

    topk_values = topk_values.numpy().squeeze()
    
    return topk_values, classes

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the validation accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes

model = load_model('checkpoint.pth')
topk_v, classes = predict('flowers/test/1/image_06743.jpg', model)    # Using a sample from test dataset

labels = [cat_to_name[str(index)] for index in classes]    # Actual names of those flowers

fig = plt.figure(figsize = (15,5))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

ax1.set_title("Input: " + cat_to_name[str(classes[0])])
imshow(process_image('flowers/test/1/image_06743.jpg'), ax = ax1);

ax2.set_xlabel("Probabilities")
sb.barplot(y = labels, x = topk_v, color = sb.color_palette()[0], ax = ax2);